<a href="https://colab.research.google.com/github/hamid-ebrahimi/YapAiTek-Assessment/blob/master/LAST_EDIT_RECOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade tensorflow-datasets

In [2]:
import tensorflow_datasets as tfds

In [3]:
!pip install -q scann tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 993.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 77.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19

In [4]:
import tensorflow_datasets as tfds
import scann

In [5]:
import tensorflow as tf

In [6]:
import  numpy as np

In [7]:
ratings_dataset, ratings_dataset_info = tfds.load(
    name='movielens/100k-ratings',
    # MovieLens dataset is not splitted into `train` and `test` sets by default.
    # So TFDS has put it all into `train` split. We load it completely and split
    # it manually.
    split='train',
    # `with_info=True` makes the `load` function return a `tfds.core.DatasetInfo`
    # object containing dataset metadata like version, description, homepage,
    # citation, etc.
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-ratings/0.1.1.incompleteU6TOA2/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


In [8]:
ratings_dataset = ratings_dataset.map(
    lambda rating: {
        'user_id': rating['user_id'],
        'raw_user_age': rating['raw_user_age'],
        'user_occupation_text': rating['user_occupation_text'],
        'movie_id': rating['movie_id'],
        'movie_title': rating['movie_title'],
        'user_rating': rating['user_rating'],
        'timestamp': rating['timestamp']
    }
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
trainset_size = 0.8 * ratings_dataset.__len__().numpy()
ratings_dataset_shuffled = ratings_dataset.shuffle(
    # the new dataset will be sampled from a buffer window of first `buffer_size`
    # elements of the dataset
    buffer_size=100_000,
    # set the random seed that will be used to create the distribution.
    seed=42,
    # `list(dataset.as_numpy_iterator()` yields different result for each call
    # Because reshuffle_each_iteration defaults to True.
    reshuffle_each_iteration=False
)
ratings_trainset = ratings_dataset_shuffled.take(trainset_size)
ratings_testset = ratings_dataset_shuffled.skip(trainset_size)
print(
    tfds.as_dataframe(ratings_trainset.take(5))
)

  movie_id                       movie_title  raw_user_age  timestamp user_id  \
0   b'143'     b'Sound of Music, The (1965)'          30.0  881108229   b'42'   
1   b'127'          b'Godfather, The (1972)'          42.0  893286501  b'683'   
2   b'258'                 b'Contact (1997)'          45.0  875945173  b'382'   
3   b'121'  b'Independence Day (ID4) (1996)'          44.0  880313603  b'666'   
4   b'506'   b'Rebel Without a Cause (1955)'          35.0  891033766  b'339'   

  user_occupation_text  user_rating  
0     b'administrator'          4.0  
1         b'librarian'          4.0  
2          b'engineer'          2.0  
3     b'administrator'          3.0  
4            b'lawyer'          4.0  


In [10]:
from pprint import pprint

for rating in ratings_trainset.take(1).as_numpy_iterator():
  pprint(rating)

{'movie_id': b'143',
 'movie_title': b'Sound of Music, The (1965)',
 'raw_user_age': 30.0,
 'timestamp': 881108229,
 'user_id': b'42',
 'user_occupation_text': b'administrator',
 'user_rating': 4.0}


In [13]:
max_timestamp = ratings_trainset.map(lambda x: x["timestamp"]).reduce(
    tf.cast(0, tf.int64), tf.maximum).numpy().max()

min_timestamp = ratings_trainset.map(lambda x: x["timestamp"]).reduce(
    np.int64(1e9), tf.minimum).numpy().min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000)

timestamp_embedding_model = tf.keras.Sequential([
  tf.keras.layers.Discretization(timestamp_buckets.tolist()),
  tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32)
])

In [14]:
for timestamp in ratings_trainset.take(1).map(lambda x: x["timestamp"]).batch(1).as_numpy_iterator():
  print(f"Timestamp embedding: {timestamp_embedding_model(timestamp)}.")

Timestamp embedding: [[-0.04561949  0.03208833 -0.00541087 -0.00137893 -0.04325727 -0.04895475
  -0.03262835  0.04294877 -0.01402198 -0.01362405  0.02683831 -0.01794656
  -0.0127156   0.02248976  0.02647914  0.01189166  0.00633897 -0.02146651
   0.03789954 -0.02615656 -0.03065565  0.02789309  0.04932951 -0.0487495
  -0.01966951  0.04213569 -0.04516125  0.02908449 -0.00227963 -0.03855907
   0.0219893  -0.02769257]].


In [15]:
  # Make a Keras StringLookup layer as the mapping (lookup)
user_id_lookup_layer = \
    tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

# StringLookup layer is a non-trainable layer and its state (the vocabulary)
# must be constructed and set before training in a step called "adaptation".
user_id_lookup_layer.adapt(
    ratings_trainset.map(
        lambda x: x['user_id']
    )
)


user_id_embedding_dim = 32
# The larger it is, the higher the capacity of the model, but the slower it is
# to fit and serve and more prone to overfitting.

user_id_embedding_layer = tf.keras.layers.Embedding(
    # Size of the vocabulary
    input_dim=user_id_lookup_layer.vocabulary_size(),
    # Dimension of the dense embedding
    output_dim=user_id_embedding_dim
)



# A model that takes raw string feature values (user_id) in and yields embeddings
user_id_model = tf.keras.Sequential(
    [
        user_id_lookup_layer,
        user_id_embedding_layer
    ]
)

In [16]:
user_id_model(tf.constant(["42"]))

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.02554625, -0.01590891, -0.04223878,  0.01323906, -0.00767599,
        -0.02467748, -0.00804744, -0.02742629,  0.04442504, -0.02347692,
        -0.01065435, -0.00772842, -0.03165432, -0.02639732, -0.03167026,
         0.01230847,  0.00689126,  0.02168624, -0.04462036,  0.01721828,
         0.02781856, -0.04924866,  0.00806018,  0.03936389,  0.04887236,
        -0.0470721 ,  0.03547288,  0.00319077, -0.04698576,  0.03894543,
         0.02417984, -0.01582661]], dtype=float32)>

In [17]:
# Make a Keras StringLookup layer as the mapping (lookup)
user_occupation_lookup_layer = \
    tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

# StringLookup layer is a non-trainable layer and its state (the vocabulary)
# must be constructed and set before training in a step called "adaptation".
user_occupation_lookup_layer.adapt(
    ratings_trainset.map(
        lambda x: x['user_occupation_text']
    )
)
user_occupation_embedding_dim = 32
# The larger it is, the higher the capacity of the model, but the slower it is
# to fit and serve and more prone to overfitting.

user_occupation_embedding_layer = tf.keras.layers.Embedding(
    # Size of the vocabulary
    input_dim=user_occupation_lookup_layer.vocabulary_size(),
    # Dimension of the dense embedding
    output_dim=user_occupation_embedding_dim
)


user_occupation_model = tf.keras.Sequential(
    [
        user_occupation_lookup_layer,
        user_occupation_embedding_layer
    ]
)

In [18]:
for x in ratings_trainset.take(3).as_numpy_iterator():
  print(f"user_occupation_text: {x['user_occupation_text']}.")

user_occupation_text: b'administrator'.
user_occupation_text: b'librarian'.
user_occupation_text: b'engineer'.


In [19]:
for user_occupation_text in ratings_trainset.take(1).map(
    lambda x: x["user_occupation_text"]
    ).batch(1).as_numpy_iterator():
  print(f"user_occupation_text embedding: {user_occupation_model(user_occupation_text)}.")

user_occupation_text embedding: [[-0.01648526 -0.00801296  0.04556822 -0.03681692 -0.01878905 -0.00316368
   0.03075888 -0.00864384  0.02980879  0.04517079 -0.03239138  0.02471434
  -0.0487654  -0.03250064 -0.01375816  0.03908279 -0.03141431  0.01611158
  -0.03115908  0.03746462  0.01210011  0.02740869 -0.00224327  0.00095705
  -0.04408373 -0.04717999 -0.02946423  0.04246554  0.0156538   0.03069749
  -0.04188316 -0.02576118]].


In [20]:
max_raw_user_age = ratings_trainset.map(lambda x: x["raw_user_age"]).reduce(
    tf.constant(0, dtype=tf.int64), lambda x, y: tf.maximum(x, tf.cast(y, tf.int64))
).numpy().max()

min_raw_user_age = ratings_trainset.map(lambda x: x["raw_user_age"]).reduce(
    tf.constant(0, dtype=tf.int64), lambda x, y: tf.minimum(x, tf.cast(y, tf.int64))
).numpy().min()


raw_user_age_buckets = np.linspace(
    min_raw_user_age, max_raw_user_age, num=1000)

raw_user_age_embedding_model = tf.keras.Sequential([
  tf.keras.layers.Discretization(raw_user_age_buckets.tolist()),
  tf.keras.layers.Embedding(len(raw_user_age_buckets) + 1, 32)
])

In [21]:
for x in ratings_trainset.take(3).as_numpy_iterator():
  print(f"raw_user_age: {x['raw_user_age']}.")

raw_user_age: 30.0.
raw_user_age: 42.0.
raw_user_age: 45.0.


In [22]:
ratings_trainset.take(1).map(
    lambda x: x["raw_user_age"]
    ).batch(1).as_numpy_iterator()

In [23]:
raw_user_age_embedding_model(32.0)

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 0.03915154,  0.04710383, -0.04937757,  0.01143893, -0.02635648,
       -0.00689157,  0.02994243,  0.01507597,  0.03272999,  0.01365653,
        0.02544412, -0.01502544,  0.00669124, -0.04279585,  0.01831818,
       -0.04700551, -0.03764707,  0.03291753, -0.00371597,  0.03175383,
        0.04313229,  0.02805389, -0.03547956, -0.00637418,  0.00043706,
        0.03528197, -0.02087778,  0.01236969,  0.00655153, -0.02240565,
       -0.00912671, -0.03622632], dtype=float32)>

In [24]:
for raw_user_age in ratings_trainset.take(1).map(
    lambda x: x["raw_user_age"]
    ).batch(1).as_numpy_iterator():
  print(f"raw_user_age embedding: {raw_user_age_embedding_model(raw_user_age)}.")

raw_user_age embedding: [[-0.03134318 -0.03358631 -0.00551931 -0.03414214 -0.02888395  0.02260259
   0.04314378  0.03310888  0.04439456  0.04739946 -0.01628642  0.03364943
  -0.01577116  0.04533983  0.02000606  0.00619598  0.04205414  0.01136844
   0.04125341 -0.01222327 -0.03912884 -0.01584556 -0.04015262  0.00137298
  -0.02905784 -0.00238725 -0.01145434  0.03810086  0.04237166 -0.021409
   0.03042383 -0.01529258]].


In [25]:
movie_id_lookup_layer = \
    tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_id_lookup_layer.adapt(
    ratings_trainset.map(
        lambda x: x['movie_id']
    )
)

# Same as user_id_embedding_dim to be able to measure the similarity
movie_id_embedding_dim = 32

movie_id_embedding_layer = tf.keras.layers.Embedding(
    input_dim=movie_id_lookup_layer.vocabulary_size(),
    output_dim=movie_id_embedding_dim)

movie_id_model = tf.keras.Sequential(
    [
        movie_id_lookup_layer,
        movie_id_embedding_layer
    ])

print(
    f"Embedding for the movie 898:\n {movie_id_model('898')}"
)

Embedding for the movie 898:
 [-0.0012697   0.03029564 -0.01699492 -0.00712071 -0.01070253  0.01282718
 -0.00357385 -0.01507262 -0.01471355  0.00729231 -0.0490065  -0.00930594
 -0.00243931  0.04790772  0.03983997  0.01545788 -0.03046303  0.03184206
 -0.02349575 -0.01939803  0.00985195 -0.01141508  0.0062559   0.03095099
  0.03270857 -0.00866408  0.00143125  0.00365802 -0.02003626  0.0058921
  0.00348804  0.02468703]


In [26]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_id_embedding = user_id_model
    self.timestamp_embedding = timestamp_embedding_model

    self.user_occupation_embedding = user_occupation_model
    self.raw_user_age_embedding = raw_user_age_embedding_model

  def call(self, inputs):

    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_id_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        self.user_occupation_embedding(inputs['user_occupation_text']),
        self.raw_user_age_embedding(inputs['raw_user_age'])

    ], axis=1)

In [27]:
user_model = UserModel()

for row in ratings_trainset.batch(1).take(1):
  print(f"Computed representations: {user_model(row)}")

Computed representations: [[-0.02554625 -0.01590891 -0.04223878  0.01323906 -0.00767599 -0.02467748
  -0.00804744 -0.02742629  0.04442504 -0.02347692 -0.01065435 -0.00772842
  -0.03165432 -0.02639732 -0.03167026  0.01230847  0.00689126  0.02168624
  -0.04462036  0.01721828  0.02781856 -0.04924866  0.00806018  0.03936389
   0.04887236 -0.0470721   0.03547288  0.00319077 -0.04698576  0.03894543
   0.02417984 -0.01582661 -0.04561949  0.03208833 -0.00541087 -0.00137893
  -0.04325727 -0.04895475 -0.03262835  0.04294877 -0.01402198 -0.01362405
   0.02683831 -0.01794656 -0.0127156   0.02248976  0.02647914  0.01189166
   0.00633897 -0.02146651  0.03789954 -0.02615656 -0.03065565  0.02789309
   0.04932951 -0.0487495  -0.01966951  0.04213569 -0.04516125  0.02908449
  -0.00227963 -0.03855907  0.0219893  -0.02769257 -0.01648526 -0.00801296
   0.04556822 -0.03681692 -0.01878905 -0.00316368  0.03075888 -0.00864384
   0.02980879  0.04517079 -0.03239138  0.02471434 -0.0487654  -0.03250064
  -0.0137581

In [28]:
movie_title_vectorization_layer = \
    tf.keras.layers.experimental.preprocessing.TextVectorization()
movie_title_vectorization_layer.adapt(
    ratings_trainset.map(
        lambda rating: rating['movie_title']
    )
)

movie_title_embedding_dim = 32
movie_title_embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(movie_title_vectorization_layer.get_vocabulary()),
    output_dim=movie_title_embedding_dim,
    # Whether or not the input value 0 is a MASK token.
    # Keras TextVectorization layer builds the vocabulary with MASK token.
    mask_zero=True
)

movie_title_model = tf.keras.Sequential(
    [
       movie_title_vectorization_layer,
       movie_title_embedding_layer,
       # each title contains multiple words, so we will get multiple embeddings
       # for each title that should be compressed into a single embedding for
       # the text. Models like RNNs, Transformers or Attentions are useful here.
       # However, averaging all the words' embeddings together is also a good
       # starting point.
       tf.keras.layers.GlobalAveragePooling1D()
    ]
)

In [29]:

class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.movieid_embedding = tf.keras.Sequential([
      movie_id_lookup_layer,
      tf.keras.layers.Embedding(movie_id_lookup_layer.vocabulary_size(), 32)
    ])
    self.title_text_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(max_tokens=max_tokens),
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per title.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])


  def call(self, inputs):
    return tf.concat([
        self.movieid_embedding(inputs["movie_id"]),
        self.title_text_embedding(inputs["movie_title"]),
    ], axis=1)

In [30]:
movie_model = MovieModel()

movie_model.title_text_embedding.layers[0].adapt(
    ratings_trainset.map(lambda x: x["movie_title"]))

for row in ratings_trainset.batch(1).take(1):
  print(f"Computed representations: {movie_model(row)}")

Computed representations: [[ 0.03395445  0.02843137  0.03357137  0.01786878  0.04395214 -0.02901204
   0.04541225 -0.01453952  0.04741726  0.01972448  0.01455661  0.02964181
   0.01512685 -0.04106736 -0.04886727 -0.04270816  0.04840846  0.04424346
  -0.00599825  0.00849841 -0.03379508  0.01336643 -0.02649258 -0.04930147
   0.03766116  0.0258449   0.02073452  0.00491334 -0.03421567 -0.0148509
  -0.03463836 -0.00502827  0.00443587  0.00820343 -0.00032027  0.01244591
  -0.01394815  0.01559198  0.00423043 -0.00580458  0.00690156  0.01187439
  -0.02351248  0.00241293 -0.00739844  0.00943154  0.00439743  0.00782892
   0.01188167  0.00824106  0.01694406 -0.00246414 -0.00166831 -0.00804042
  -0.02420885 -0.01624595  0.00685804  0.00392953 -0.00834749  0.00016173
  -0.00600031  0.00249689  0.01867597  0.02247198]]


In [31]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = user_model

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [32]:
query_model = QueryModel([32])

for row in ratings_trainset.batch(1).take(1):
  print(f"Computed representations: {query_model(row)}")

Computed representations: [[ 0.01202109  0.00537161 -0.01514304  0.02014583 -0.04326352 -0.04781391
   0.03328235 -0.06458955  0.0152151   0.00743057 -0.02321709  0.00383867
  -0.04668204  0.04154115 -0.00116381  0.09216073  0.02409613  0.01501766
  -0.00182456 -0.03335384  0.02602167 -0.00445136  0.02925517 -0.04385383
   0.0445489  -0.00634001 -0.00492207 -0.01961044 -0.04181016 -0.02204255
   0.04947113 -0.00121139]]


In [33]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = movie_model

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)


candidate_model = CandidateModel([32])


In [34]:
candidate_model = CandidateModel([32])

for row in ratings_trainset.batch(1).take(1):
  print(f"Computed representations: {candidate_model(row)}")

Computed representations: [[-6.20199041e-03  4.46925275e-02  1.03523098e-02 -3.77633143e-04
  -1.57815721e-02  1.31567605e-02 -4.90116235e-03  3.17771453e-04
  -8.08759034e-03 -3.39068696e-02  5.62363956e-03  9.47601278e-04
  -2.34467033e-02  4.11855057e-02 -7.36210821e-03 -7.77841359e-03
  -9.50595923e-03 -9.80145996e-05 -1.51087875e-02  4.63184807e-03
  -1.36168431e-02  8.98081996e-03 -4.26250026e-02 -2.19290089e-02
   6.50926530e-02  7.25661730e-03  1.47394370e-02 -2.58456543e-02
  -9.98951960e-03 -3.35572772e-02  9.45700333e-03 -1.79955631e-03]]


In [35]:
# We don't need rating field for the retrieval task
retrieval_ratings_trainset = ratings_trainset.map(
    lambda rating: {
        'user_id': rating['user_id'],
        'raw_user_age': rating['raw_user_age'],
        'user_occupation_text': rating['user_occupation_text'],
        'movie_id': rating['movie_id'],
        'movie_title': rating['movie_title'],
        'timestamp': rating['timestamp']
    }
)

retrieval_ratings_testset = ratings_testset.map(
    lambda rating: {
        'user_id': rating['user_id'],
        'raw_user_age': rating['raw_user_age'],
        'user_occupation_text': rating['user_occupation_text'],
        'movie_id': rating['movie_id'],
        'movie_title': rating['movie_title'],
        'timestamp': rating['timestamp']
    }
)

In [36]:
movies_dataset, movies_dataset_info = tfds.load(
    name='movielens/100k-movies',
    split='train',
    with_info=True
)

print(
    tfds.as_dataframe(movies_dataset.take(5), movies_dataset_info)
)


# We are using just `movie_id` feature for making the candidates representation
candidates_corpus_dataset = movies_dataset.map(
    lambda movie: {'movie_id':movie['movie_id'],
                   'movie_title': movie['movie_title']
                   }
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete4UJTLN/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.
  movie_genres movie_id                      movie_title
0          [4]  b'1681'           b'You So Crazy (1994)'
1       [4, 7]  b'1457'   b'Love Is All There Is (1996)'
2       [1, 3]   b'500'          b'Fly Away Home (1996)'
3          [0]   b'838'  b'In the Line of Duty 2 (1987)'
4          [7]  b'1648'       b'Niagara, Niagara (1997)'


In [37]:
import tensorflow_recommenders as tfrs

factorized_top_k_metrics = tfrs.metrics.FactorizedTopK(
    # dataset of candidate embeddings from which candidates should be retrieved
    candidates=candidates_corpus_dataset.batch(128).map(
        candidate_model
    )
)

In [38]:
retrieval_task_layer = tfrs.tasks.Retrieval(
    metrics=factorized_top_k_metrics
)

In [39]:
class RetrievalModel(tfrs.models.Model):
  """MovieLens candidate generation model"""

  def __init__(self, query_model, candidate_model, retrieval_task_layer):
    super().__init__()
    self.query_model: tf.keras.Model = query_model
    self.candidate_model: tf.keras.Model = candidate_model
    self.retrieval_task_layer: tf.keras.layers.Layer = retrieval_task_layer

 #def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
  def compute_loss(self, features, training=False) -> tf.Tensor:
    query_embeddings = self.query_model(({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
        'raw_user_age': features['raw_user_age'],
        'user_occupation_text': features['user_occupation_text']
    }))
    positive_candidate_embeddings = self.candidate_model(({
        'movie_id': features['movie_id'],
        'movie_title': features['movie_title']
    }))

    loss = self.retrieval_task_layer(
        query_embeddings,
        positive_candidate_embeddings
        # ,compute_metrics=not training  # To speed up training
    )
    return loss

In [40]:
movielens_retrieval_model = RetrievalModel(
    query_model,
    candidate_model,
    retrieval_task_layer
)

optimizer_step_size = 0.01
movielens_retrieval_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(
        learning_rate=optimizer_step_size
    )
)

In [41]:
retrieval_cached_ratings_trainset = \
  retrieval_ratings_trainset.shuffle(100_000).batch(8192).cache()
retrieval_cached_ratings_testset = \
  retrieval_ratings_testset.batch(4096).cache()

num_epochs = 2
history = movielens_retrieval_model.fit(
    retrieval_cached_ratings_trainset,
    validation_data=retrieval_cached_ratings_testset,
    validation_freq=1,
    epochs=num_epochs
)

Epoch 1/2
10/10 [==============================] - 54s 5s/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0109 - factorized_top_k/top_50_categorical_accuracy: 0.0520 - factorized_top_k/top_100_categorical_accuracy: 0.0973 - loss: 70310.7848 - regularization_loss: 0.0000e+00 - total_loss: 70310.7848 - val_factorized_top_k/top_1_categorical_accuracy: 0.0022 - val_factorized_top_k/top_5_categorical_accuracy: 0.0109 - val_factorized_top_k/top_10_categorical_accuracy: 0.0218 - val_factorized_top_k/top_50_categorical_accuracy: 0.0960 - val_factorized_top_k/top_100_categorical_accuracy: 0.1702 - val_loss: 29480.2969 - val_regularization_loss: 0.0000e+00 - val_total_loss: 29480.2969
Epoch 2/2
10/10 [==============================] - 45s 5s/step - factorized_top_k/top_1_categorical_accuracy: 0.0017 - factorized_top_k/top_5_categorical_accuracy: 0.0114 - factorized_top_k/top_10_

In [42]:
def generate_candidate_tensors():
    for item in candidates_corpus_dataset:
        yield {
            "movie_id": tf.expand_dims(item["movie_id"], axis=0),
            "movie_title": tf.expand_dims(item["movie_title"], axis=0)
        }

candidate_tensors = tf.data.Dataset.from_generator(
    generate_candidate_tensors,
    output_signature={
        "movie_id": tf.TensorSpec(shape=(None,), dtype=tf.string),
        "movie_title": tf.TensorSpec(shape=(None,), dtype=tf.string)
    }
)


In [43]:
brute_force_layer = tfrs.layers.factorized_top_k.BruteForce(
    movielens_retrieval_model.query_model
)

candidate_id_dataset = candidates_corpus_dataset.map(lambda x: x["movie_id"])

candidate_tensors = candidates_corpus_dataset.map(lambda x: {
    "movie_id": x["movie_id"],
    "movie_title": x["movie_title"]
})

brute_force_layer.index_from_dataset(
    tf.data.Dataset.zip(
        (
            candidate_id_dataset.batch(100),
            candidate_tensors.batch(100).map(candidate_model)
        )
    )
)

In [44]:
user_info = {
    'user_id': tf.constant(["12345"]),
    'timestamp': tf.constant([1609459200], dtype=tf.int32),
    'raw_user_age': tf.constant([25]),
    'user_occupation_text': tf.constant(['Engineer'])
}

In [45]:
user_info

{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'12345'], dtype=object)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1609459200], dtype=int32)>,
 'raw_user_age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([25], dtype=int32)>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Engineer'], dtype=object)>}

In [46]:
afinity_scores, movie_ids = brute_force_layer(user_info)

In [ ]:
print(f"Recommendations for user {user_info} using brute_force: {movie_ids[0, :10]}")

Recommendations for user {'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'12345'], dtype=object)>, 'timestamp': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1609459200], dtype=int32)>, 'raw_user_age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([25], dtype=int32)>, 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Engineer'], dtype=object)>} using brute_force: [b'876' b'406' b'833' b'975' b'829' b'827' b'358' b'295' b'877' b'597']


In [ ]:
movie_ids[0, :10]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'829', b'682', b'876', b'833', b'877', b'350', b'263', b'1013',
       b'975', b'330'], dtype=object)>

In [ ]:
afinity_scores[0, :10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1.0429497 , 1.0287812 , 1.0242292 , 1.0154908 , 1.008818  ,
       1.002595  , 0.9562249 , 0.95454353, 0.9441894 , 0.8942301 ],
      dtype=float32)>

In [ ]:
ratings_trainset

<TakeDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'raw_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [47]:
class RankingModel(tfrs.models.Model):
  """MovieLens ranking model"""

  def __init__(self, query_model, candidate_model):
    super().__init__()

    self.query_model: tf.keras.Model = query_model
    self.candidate_model: tf.keras.Model = candidate_model
    self.rating_model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ]
    )
    self.ranking_task_layer: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )


  def compute_loss(self, features, training=False) -> tf.Tensor:
    query_embeddings = self.query_model(({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
        'raw_user_age': features['raw_user_age'],
        'user_occupation_text': features['user_occupation_text']
    }))
    candidate_embeddings = self.candidate_model(({
        'movie_id': features['movie_id'],
        'movie_title': features['movie_title']
    }))
    rating_predictions = self.rating_model(
        tf.concat(
            [query_embeddings, candidate_embeddings],
            axis=1
        )
        # We could use `tf.keras.layers.Concatenate(axis=1)([x, y])`
    )

    loss = self.ranking_task_layer(
        predictions=rating_predictions,
        labels=features["user_rating"]
    )
    return loss

In [48]:
movielens_ranking_model = RankingModel(query_model, candidate_model)

optimizer_step_size = 0.1
movielens_ranking_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(
        learning_rate=optimizer_step_size
    )
)

In [49]:
ranking_ratings_trainset = ratings_trainset.shuffle(100_000).batch(8192).cache()
ranking_ratings_testset = ratings_testset.batch(4096).cache()

history = movielens_ranking_model.fit(
    ranking_ratings_trainset,
    validation_data=ranking_ratings_testset,
    validation_freq=1,
    epochs=5
)

Epoch 1/5
10/10 [==============================] - 14s 839ms/step - root_mean_squared_error: 3.0836 - loss: 9.0368 - regularization_loss: 0.0000e+00 - total_loss: 9.0368 - val_root_mean_squared_error: 3.0075 - val_loss: 9.1562 - val_regularization_loss: 0.0000e+00 - val_total_loss: 9.1562
Epoch 2/5
10/10 [==============================] - 2s 207ms/step - root_mean_squared_error: 1.6763 - loss: 2.6386 - regularization_loss: 0.0000e+00 - total_loss: 2.6386 - val_root_mean_squared_error: 1.1785 - val_loss: 1.4216 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.4216
Epoch 3/5
10/10 [==============================] - 2s 207ms/step - root_mean_squared_error: 1.1181 - loss: 1.2397 - regularization_loss: 0.0000e+00 - total_loss: 1.2397 - val_root_mean_squared_error: 1.0883 - val_loss: 1.2103 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.2103
Epoch 4/5
10/10 [==============================] - 1s 108ms/step - root_mean_squared_error: 1.0634 - loss: 1.1277 - regularization_l

In [50]:
ranking_ratings_trainset

<CacheDataset element_spec={'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'raw_user_age': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'user_occupation_text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'movie_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}>

In [ ]:
new_sample = {
    "user_id": 123,
    "timestamp": 456,
    "raw_user_age": 30,
    "user_occupation_text": "engineer",
    "movie_id": 789,
    "movie_title": "New Movie"
}

In [56]:
sample = {
    'user_id': '42',
    'raw_user_age': 30,
    'user_occupation_text': 'administrator',
    'movie_id': '143',
    'movie_title': 'Sound of Music, The (1965)',
    'timestamp': 881108229
}

movielens_ranking_model.predict(ranking_ratings_testset)


NotImplementedError: ignored